In [5]:
import zipfile
from datetime import datetime
import os
import requests
import pandas as pd
import logging
import logging.config
import json
import shutil

# Load configurations
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Configure logging
logging.config.fileConfig(config['log_config'])
logger = logging.getLogger('root')

# URL of the file to be downloaded
url = config['download_url']

# Current date in YYYY-MM-DD format
current_date = datetime.now().strftime("%m.%d.%Y")

# Define the downloaded filename for the ZIP file
filename = f'../Monthly_Input_Files/data_{current_date}.zip'

# Define the extracted file path
extract_path = f'../Monthly_Input_Files/data_{current_date}'

try:
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()
    logger.info(f"Downloading file '{filename}'.")

        # Write the content of the response to a file
    with open(filename, 'wb') as file:
        file.write(response.content)
    logger.info(f"File '{filename}' downloaded successfully!")

    # Check if the extract folder already exists and has files
    if os.path.exists(extract_path):
        shutil.rmtree(extract_path)

    # Create a new extract folder
    os.makedirs(extract_path, exist_ok=True)

        # Extract the ZIP file, overwrite any existing files
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    logger.info(f"Files extracted to '{extract_path}'.")

except requests.exceptions.RequestException as e:
    logger.error("Error occurred while downloading or extracting the file: " + str(e))

# Process the files in the extract folder
for extracted_file in os.listdir(extract_path):
    file_path = os.path.join(extract_path, extracted_file)
    if extracted_file.endswith(('.xls', '.xlsx', '.xlsm')):
        new_name = f'Medi-Cal Rates {current_date}.xlsx'
        new_path = os.path.join(extract_path, new_name)
        # Check if the new filename already exists and rename the file if it does not
        logger.info(f"File {new_name} already exists. No need to rename.") if os.path.exists(new_path) else (os.rename(file_path, new_path), logger.info(f"Excel file found and renamed to: {new_name}"))


# Check if a file path was assigned and process data accordingly
if 'new_path' in locals():
    date_part = new_path.split(" ")[-1].split(".")  
    split_date = '-'.join(date_part[0:3])
    month, day, year = split_date.split('-')
    date = f'{year}-{month}-01'
    df = pd.read_excel(new_path, skiprows=1)
    df_ac= pd.read_excel("../Augment_Codes/Augment Codes.xlsx", skiprows=1)
    
    # Define new column names based on the data structure in your file    
    df.columns = config['columns_names'] 
    df_ac.columns = config['ac_columns_names']
    # Remove the $ sign and convert the column to numeric, coerce errors to NaN
    df['Basic_Rate'] = pd.to_numeric(df['Basic_Rate'].str.replace('[\$,]', '', regex=True), errors='coerce').round(2)
    df['Child_Rate'] = pd.to_numeric(df['Child_Rate'].str.replace('[\$,]', '', regex=True), errors='coerce').round(2)
    df['ER_Rate'] = pd.to_numeric(df['ER_Rate'].str.replace('[\$,]', '', regex=True), errors='coerce').round(2)
    df['Rental_Rate'] = pd.to_numeric(df['Rental_Rate'].str.replace('[\$,]', '', regex=True), errors='coerce').round(2)
    df['Prof_%'] = pd.to_numeric(df['Prof_%'], errors='coerce').round(2)

    # Define functions to calculate rates for each Proc_Type
    def Purchase_Calculation(df,proc_type):
        filtered_df_1 = df.copy()
        filtered_df_1 = filtered_df_1[filtered_df_1['Basic_Rate'] > 0.01]
        filtered_df_1['FEE'] = filtered_df_1['Basic_Rate'].round(2)
        filtered_df_1['MODIFIER_1']= config['modifier_purchase']
        filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
        filtered_df_1['AGE_UNIT']=''
        filtered_df_1['FROM_AGE']=''
        filtered_df_1['THROUGH_AGE']=''
        filtered_df_1['PLACE_OF_SERVICE_CODE']=''
    
        final_df_a = filtered_df_1[config['hrp_columns_format']]
        return final_df_a

    # Define functions to calculate rates for each Proc_Type
    def Rental_Calculation(df,proc_type):
        filtered_df_1 = df.copy()

        filtered_df_1['FEE']=filtered_df_1['Rental_Rate'].round(2)
        filtered_df_1['MODIFIER_1']= config['modifier_rental']
        filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
        filtered_df_1['AGE_UNIT']=''
        filtered_df_1['FROM_AGE']=''
        filtered_df_1['THROUGH_AGE']=''
        filtered_df_1['PLACE_OF_SERVICE_CODE']=''
        final_df_a = filtered_df_1[config['hrp_columns_format']]     
        return final_df_a

    def Basic_Rate_Calculation(df,proc_type):
        filtered_df_1 = df[(df['Basic_Rate'] > 0)].copy()
        filtered_df_2 = df[(df['Basic_Rate'] > 0) & (df['Prof_%'] > 0)].copy()
        filtered_df_3 = df[(df['Basic_Rate'] > 0) & (df['Prof_%'] !=0) & (1- df['Prof_%'] > 0)].copy()

        filtered_df_1['FEE']=filtered_df_1['Basic_Rate'].round(2)
        filtered_df_1['MODIFIER_1']=''
        filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
        # Create a professional rate DataFrame
        prof_df = filtered_df_2.copy()
        prof_df['FEE'] = (prof_df['Basic_Rate'] * prof_df['Prof_%']).round(2)
        prof_df['MODIFIER_1'] = config['modifier_professional']
        prof_df['SVC_CD']=prof_df['Proc_Code']
 
        # Create a technical rate DataFrame
        tech_df = filtered_df_3.copy()
        tech_df['FEE'] = (tech_df['Basic_Rate'] * (1 - tech_df['Prof_%'])).round(2)
        tech_df['MODIFIER_1'] = config['modifier_technical']
        tech_df['SVC_CD']=tech_df['Proc_Code']
 
        # Append the technical DataFrame to the professional DataFrame
        result_df = pd.concat([filtered_df_1,prof_df, tech_df], ignore_index=True)
        result_df['AGE_UNIT']=''
        result_df['FROM_AGE']=''
        result_df['THROUGH_AGE']=''
        result_df['PLACE_OF_SERVICE_CODE']=''
        final_df_a = result_df[config['hrp_columns_format']]
    
        return final_df_a

    def Child_Rate_Calculation(df,proc_type):
        # Filter DataFrame for rows where both Prof_% and Basic_Rate are greater than 0
        filtered_df_1 = df[(df['Proc_Type']==proc_type) & (df['Child_Rate'] > 0)].copy()
        filtered_df_2 = df[(df['Proc_Type']==proc_type) & (df['Child_Rate'] > 0) & (df['Prof_%'] > 0)].copy()
        filtered_df_3 = df[(df['Proc_Type']==proc_type) & (df['Child_Rate'] > 0) & (df['Prof_%'] !=0) & (1- df['Prof_%'] > 0)].copy()
        result_df=pd.DataFrame()
        if len(filtered_df_1)>0:
            filtered_df_1['FEE']=filtered_df_1['Child_Rate'].round(2)
            filtered_df_1['MODIFIER_1']=''
            filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
            
            # Create a professional rate DataFrame
            prof_df = filtered_df_2.copy()
            prof_df['FEE'] = (prof_df['Child_Rate'] * prof_df['Prof_%']).round(2)
            prof_df['MODIFIER_1'] = config['modifier_professional']
            prof_df['SVC_CD']=prof_df['Proc_Code']
 
            # Create a technical rate DataFrame
            tech_df = filtered_df_3.copy()
            tech_df['FEE'] = (tech_df['Child_Rate'] * (1 - tech_df['Prof_%'])).round(2)
            tech_df['MODIFIER_1'] = config['modifier_technical']
            tech_df['SVC_CD']=tech_df['Proc_Code']
    
            # Append the technical DataFrame to the professional DataFrame
            result_df = pd.concat([filtered_df_1,prof_df, tech_df], ignore_index=True)
            result_df['AGE_UNIT']=config["age_unit_child"]
            result_df['FROM_AGE']=config["from_age_child"]
            result_df['THROUGH_AGE']=config["through_age_child"]
            result_df['PLACE_OF_SERVICE_CODE']=''
            result_df = result_df[config['hrp_columns_format']]
        return result_df

    def ER_Rate_Calculation(df,proc_type):
        # Filter DataFrame for rows where both Prof_% and Basic_Rate are greater than 0
        filtered_df_1 = df[(df['Proc_Type']==proc_type) & (df['ER_Rate'] > 0) & (df['ER_Ind']>0)].copy()
        filtered_df_2 = df[(df['Proc_Type']==proc_type) & (df['ER_Rate'] > 0) & (df['Prof_%'] > 0) & (df['ER_Ind']>0)].copy()
        filtered_df_3 = df[(df['Proc_Type']==proc_type) & (df['ER_Rate'] > 0) & (df['Prof_%'] !=0) & (1- df['Prof_%'] > 0) & (df['ER_Ind']>0)].copy()
        
        filtered_df_1['FEE']=filtered_df_1['ER_Rate'].round(2)
        filtered_df_1['MODIFIER_1']=''
        filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
        # Create a professional rate DataFrame
        prof_df = filtered_df_2.copy()
        prof_df['FEE'] = (prof_df['ER_Rate'] * prof_df['Prof_%']).round(2)
        prof_df['MODIFIER_1'] = config['modifier_professional']
        prof_df['SVC_CD']=prof_df['Proc_Code']
    
        # Create a technical rate DataFrame
        tech_df = filtered_df_3.copy()
        tech_df['FEE'] = (tech_df['ER_Rate'] * (1 - tech_df['Prof_%'])).round(2)
        tech_df['MODIFIER_1'] = config['modifier_technical']
        tech_df['SVC_CD']=tech_df['Proc_Code']
    
        # Append the technical DataFrame to the professional DataFrame
        result_df = pd.concat([filtered_df_1,prof_df, tech_df], ignore_index=True)
        result_df['AGE_UNIT']=''
        result_df['FROM_AGE']=''
        result_df['THROUGH_AGE']=''
        result_df['PLACE_OF_SERVICE_CODE']= config['place_of_service_code']
        return result_df[config['hrp_columns_format']]

    def anes_calculation(df,proc_type):
        # Filter DataFrame for rows where both Prof_% and Basic_Rate are greater than 0
        filtered_df_1 = df.copy()
        
        filtered_df_1['CPT Code']=filtered_df_1['Proc_Code']
        filtered_df_1['CPT Descriptor']=filtered_df_1['Procedure_Description']
        filtered_df_1['Base Unit Value']=filtered_df_1['Unit_Value']
        
        filtered_df_1['AGE_UNIT']=''
        filtered_df_1['FROM_AGE']=''
        filtered_df_1['THROUGH_AGE']=''
        filtered_df_1['PLACE_OF_SERVICE_CODE']=''
        final_df_a = filtered_df_1[config['anes_hrp_format']]
        
        return final_df_a
    
    # Create a dictionary to hold dataframes for each a_type
    df_dict = {}

    # Get unique Proc_Types
    Proc_Types = df['Proc_Type'].unique()

    # Create a dataframe for each Proc_Types and store it in the dictionary
    for Proc_Type in Proc_Types:
        df_dict[Proc_Type] = df[df['Proc_Type'] == Proc_Type]

    # Initialize a new dictionary to hold the filtered dataframes
    filtered_df_dict = {}

    # Define the directory path to include today's date
    output_files_folder_path = f'../Monthly_Output_Files/{date}/'

    # Create the directory if it does not already exist
    os.makedirs(output_files_folder_path, exist_ok=True)


    # Filter the dataframe for Proc_Type 'I' and Proc_Code starting with 'C','J','S'
    if 'I' in df_dict and df_dict['I']['Proc_Code'].str.startswith(('C','J','S')).any():
            filtered_df_dict = df_dict['I'][df_dict['I']['Proc_Code'].str.startswith(('C','J','S'))].copy()
            filtered_df_dict['Basic_Rate'] = filtered_df_dict['Basic_Rate'] - 4.46
            filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0.01)]
            Basic_df=Basic_Rate_Calculation(filtered_df_dict,'I')
            Child_df=Child_Rate_Calculation(filtered_df_dict,'I')
            ER_df =ER_Rate_Calculation(filtered_df_dict,'I')
            final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
            final_df.to_csv(f'{output_files_folder_path}Mcal_Injections.{date}.txt', sep='|', index=False)
            logging.info(f"File 'Mcal_Injections.{date}.txt' created successfully!")
            
    # Filter the dataframe for Proc_Type 'I' and Proc_Code starting with 'A9','P9','Q'
    if 'I' in df_dict and  df_dict['I']['Proc_Code'].str.startswith(('A9','P9','Q')).any():
            filtered_df_dict = df_dict['I'][df_dict['I']['Proc_Code'].str.startswith(('A9','P9','Q'))].copy()
            filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
            Basic_df=Basic_Rate_Calculation(filtered_df_dict,'I')
            Child_df=Child_Rate_Calculation(filtered_df_dict,'I')
            ER_df =ER_Rate_Calculation(filtered_df_dict,'I')
            final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
            final_df.to_csv(f'{output_files_folder_path}Mcal_Injections_Other.{date}.txt', sep='|', index=False)
            logging.info(f"File 'Mcal_Injections_Other.{date}.txt' created successfully!")
            
    # Filter the dataframe for Proc_Type 'I' and Proc_Code starting with '90'
    if 'I' in df_dict and  df_dict['I']['Proc_Code'].str.startswith(('90')).any():
            filtered_df_dict = df_dict['I'][df_dict['I']['Proc_Code'].str.startswith(('90'))].copy()
            filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
            Basic_df=Basic_Rate_Calculation(filtered_df_dict,'I')
            Child_df=Child_Rate_Calculation(filtered_df_dict,'I')
            ER_df =ER_Rate_Calculation(filtered_df_dict,'I')
            final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
            final_df.to_csv(f'{output_files_folder_path}Mcal_Vaccines.{date}.txt', sep='|', index=False)
            logging.info(f"File 'Mcal_Vaccines.{date}.txt' created successfully!")
    
    # Filter the dataframe for Proc_Type 'J'
    if 'J' in df_dict:
        filtered_df_dict = df_dict['J'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        anes_df=anes_calculation(filtered_df_dict,'J')
        
        final_df = anes_df
        final_df.to_csv(f'{output_files_folder_path}Anesthesia_Base.{date}.txt', sep=',', index=False)
        logging.info(f"File 'Anesthesia_Base.{date}.txt' created successfully!")

    # Filter the dataframe for Proc_Type 'K'
    if 'K' in df_dict:
        filtered_df_dict = df_dict['K'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'K')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'K')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'K')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_Surgical.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_Surgical.{date}.txt' created successfully!")
    
    # Filter the dataframe for Proc_Type 'L'
    if 'L' in df_dict:
        filtered_df_dict = df_dict['L'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'L')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'L')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'L')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_Radiology.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_Radiology.{date}.txt' created successfully!")
    
    # Filter the dataframe for Proc_Type 'M'
    if 'M' in df_dict:
        filtered_df_dict = df_dict['M'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'M')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'M')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'M')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_Path and Lab.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_Path and Lab.{date}.txt' created successfully!")

    # Filter the dataframe for Proc_Type 'N' and 'F'   
    if 'N' or 'F' in df_dict:
        filtered_df_dict_N = df_dict['N'].copy()
        filtered_df_dict_F = df_dict['F'].copy()
        filtered_df_dict = pd.concat([filtered_df_dict_N,filtered_df_dict_F], ignore_index=True)
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'N')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'N')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'N')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_Medicine.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_Medicine.{date}.txt' created successfully!")

    # Filter the dataframe for Proc_Type 'O' 
    if 'O' in df_dict:
        filtered_df_dict = df_dict['O'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'O')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'O')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'O')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_Asst_Surgical.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_Asst_Surgical.{date}.txt' created successfully!")

    # Filter the dataframe for Proc_Type 'P'   
    if 'P' in df_dict:
        filtered_df_dict = df_dict['P'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'P')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'P')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'P')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_Podiatrists.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_Podiatrists.{date}.txt' created successfully!")

    # Filter the dataframe for Proc_Type 'Q'    
    if 'Q' in df_dict:
        filtered_df_dict = df_dict['Q'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'Q')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'Q')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'Q')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_Behavioral Health.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_Behavioral Health.{date}.txt' created successfully!")
    
    # Filter the dataframe for Proc_Type 'T'
    if 'T' in df_dict:
        filtered_df_dict = df_dict['T'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'T')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'T')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'T')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_EPSDT.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_EPSDT.{date}.txt' created successfully!")
    
    # Filter the dataframe for Proc_Type '3'
    if '3' in df_dict:
        filtered_df_dict = df_dict['3'].copy()
        filtered_df_dict = filtered_df_dict[(filtered_df_dict['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict,'3')
        Child_df=Child_Rate_Calculation(filtered_df_dict,'3')
        ER_df =ER_Rate_Calculation(filtered_df_dict,'3')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}MCal_Vision.{date}.txt', sep='|', index=False)
        logging.info(f"File 'MCal_Vision.{date}.txt' created successfully!")
    
    # Filter the dataframe for Proc_Type '1'
    if '1' in df_dict:
        filtered_df_dict_1 = df_dict['1'][(~df_dict['1']['Proc_Code'].str.startswith('A0')) & (df_dict['1']['Rental_Rate'] == 0)].copy()
        filtered_df_dict_1 = filtered_df_dict_1[(filtered_df_dict_1['Basic_Rate'] > 0)]
        Basic_df=Basic_Rate_Calculation(filtered_df_dict_1,'1')
        Child_df=Child_Rate_Calculation(filtered_df_dict_1,'1')
        ER_df =ER_Rate_Calculation(filtered_df_dict_1,'1')
        final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
        final_df.to_csv(f'{output_files_folder_path}Mcal_Allied.{date}.txt', sep='|', index=False)
        logging.info(f"File 'Mcal_Allied.{date}.txt' created successfully!")

        filtered_df_dict_2 = df_dict['1'][(~df_dict['1']['Proc_Code'].str.startswith('A0')) & (df_dict['1']['Rental_Rate'] > 0)].copy()
        Purchase_df = Purchase_Calculation(filtered_df_dict_2,'1')
        Rental_df = Rental_Calculation(filtered_df_dict_2,'1')
        final_df_2 = pd.concat([Purchase_df,Rental_df], ignore_index=True)
        final_df_2.to_csv(f'{output_files_folder_path}Mcal_DME.{date}.txt', sep='|', index=False)
        logging.info(f"File 'Mcal_DME.{date}.txt' created successfully!")

    else:
        logging.error("File path not found.")
    

    merged_df = pd.merge(df, df_ac, how='right', on=['Proc_Code', 'Proc_Type'])
    merged_df = merged_df[merged_df['Proc_Type']!= 'J']
    df_augment = merged_df
    filtered_aug = df_augment[(df_augment['Basic_Rate'] > 0)].copy()
    filtered_aug_2 = df_augment[(df_augment['Basic_Rate'] > 0) & (df_augment['Prof_%'] !=0) & (1- df_augment['Prof_%'] > 0)].copy()
    
    filtered_aug['FEE']=(filtered_aug['Basic_Rate']*1.4344).round(2)
    filtered_aug['MODIFIER_1']=''
    filtered_aug['SVC_CD']=filtered_aug['Proc_Code']
    
    # Create a technical rate DataFrame
    tech_df = filtered_aug_2.copy()
    tech_df['FEE'] = (tech_df['Basic_Rate'] * (1 - tech_df['Prof_%']) * 1.4344).round(2)
    tech_df['MODIFIER_1'] = 'TC'
    tech_df['SVC_CD']=tech_df['Proc_Code']
 
    # Append the technical DataFrame to the professional DataFrame
    result_df = pd.concat([filtered_aug, tech_df], ignore_index=True)
    result_df['AGE_UNIT']=''
    result_df['FROM_AGE']=''
    result_df['THROUGH_AGE']=''
    result_df['PLACE_OF_SERVICE_CODE']=''
    final_df_a = result_df[['SVC_CD', 'MODIFIER_1', 'FEE']]
    final_df_a.to_csv(f'{output_files_folder_path}/DNU_Mcal_Augment.{date}.txt', sep='|', index=False)
    logging.info(f"File 'DNU_Mcal_Augment.{date}.txt' created successfully!")


